In [ ]:
# ------------------------------------------------------------------------
#
# TITLE - fit_dens_gse.ipynb
# AUTHOR - James Lane
# PROJECT - ges-mass
#
# ------------------------------------------------------------------------
#
# Docstrings and metadata:
'''Fit density profiles to GS/E Gaia/APOGEE halo data
'''

__author__ = "James Lane"

In [ ]:
### Imports

# Basic
import os, sys, copy, dill as pickle
import numpy as np

# Matplotlib and plotting 
import matplotlib.pyplot as plt

# Project specific
sys.path.insert(0,'../../src/')
from ges_mass import mass as pmass
from ges_mass import densprofiles as pdens
from ges_mass import util as putil

### Notebook setup

%matplotlib inline
plt.style.use('../../src/mpl/project.mplstyle') # This must be exactly here
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

### Keywords, Pathing, Loading, Data Preparation

In [ ]:
# %load ../../src/nb_modules/keywords_pathing_loading_data_prep.py
## Keywords
cdict = putil.load_config_to_dict()
keywords = ['BASE_DIR','APOGEE_DR','APOGEE_RESULTS_VERS','GAIA_DR','NDMOD',
            'DMOD_MIN','DMOD_MAX','LOGG_MIN','LOGG_MAX','FEH_MIN','FEH_MAX',
            'FEH_MIN_GSE','FEH_MAX_GSE','DF_VERSION','KSF_VERSION','NPROCS',
            'RO','VO','ZO']
base_dir,apogee_dr,apogee_results_vers,gaia_dr,ndmod,dmod_min,dmod_max,\
    logg_min,logg_max,feh_min,feh_max,feh_min_gse,feh_max_gse,df_version,\
    ksf_version,nprocs,ro,vo,zo = putil.parse_config_dict(cdict,keywords)
logg_range = [logg_min,logg_max]
feh_range = [feh_min,feh_max]
feh_range_gse = [feh_min_gse,feh_max_gse]
feh_range_all = [feh_min,feh_max_gse]
feh_range_fit = copy.deepcopy(feh_range_gse) # Need to choose here

## Pathing
fit_paths = putil.prepare_paths(base_dir,apogee_dr,apogee_results_vers,gaia_dr,
                                df_version,ksf_version)
data_dir,version_dir,ga_dir,gap_dir,df_dir,ksf_dir,fit_dir = fit_paths

## Filenames
fit_filenames = putil.prepare_filenames(ga_dir,gap_dir,feh_range_gse)
apogee_SF_filename,apogee_effSF_filename,apogee_effSF_mask_filename,\
    iso_grid_filename,clean_kinematics_filename = fit_filenames

## File loading and data preparation
fit_stuff,other_stuff = putil.prepare_fitting(fit_filenames,
    [ndmod,dmod_min,dmod_max],ro,zo,return_other=True)
apogee_effSF_mask,dmap,iso_grid,jkmins,dmods,ds,effsel_grid,apof,\
    allstar_nomask,orbs_nomask = fit_stuff
Rgrid,phigrid,zgrid = effsel_grid
# apogee_SF,apogee_effSF_grid_inclArea,apogee_effSF_grid_inclArea_Jac = other_stuff

# ## Load the distribution functions
# df_filename = df_dir+'dfs.pkl'
# betas = [0.3,0.9]
# dfs = putil.load_distribution_functions(df_filename, betas)

# ## Load the APOGEE field information, can also similarly load 
# ## '...apogee_field_glons.npy', '...apogee_field_glats.npy', 
# ## '...apogee_field_location_ids.npy'
# apogee_fields = np.load(ga_dir+'apogee_fields.npy')

### Global Parameters

In [ ]:
# %load ../../src/nb_modules/global_fitting_params.py
## general kwargs
verbose = True

## HaloFit kwargs (ordering follows HaloFit.__init__)
# allstar and orbs loaded in prep cell
init = None
init_type = 'ML'
# fit_type provided at runtime
mask_disk = True
mask_halo = True
# densfunc, selec provided at runtime
# effsel, effsel_grid, effsel_mask, dmods loaded in prep cell
nwalkers = 100
nit = int(1e4)
ncut = int(1e3)
# usr_log_prior provided at runtime
n_mass = 5000 # int(nwalkers*(nit-ncut))
int_r_range = [2.,70.]
iso = None # Will read from iso_grid_filename
# iso_filename, jkmins loaded in prep cell
# feh_range provided at runtime
# logg_range loaded in config cell
# fit_dir, gap_dir, ksf_dir loaded in prep cell
# version provided at runtime
# ro, vo, zo loaded in config cell

hf_kwargs = {## HaloFit parameters
             'allstar':allstar_nomask,
             'orbs':orbs_nomask,
             'init':init,
             'init_type':init_type,
             # 'fit_type':fit_type, # provided at runtime
             'mask_disk':mask_disk,
             'mask_halo':mask_halo,
             ## _HaloFit parameters
             # 'densfunc':densfunc, # provided at runtime
             # 'selec':selec, # provided at runtime
             'effsel':apof,
             'effsel_mask':apogee_effSF_mask,
             'effsel_grid':effsel_grid,
             'dmods':dmods,
             'nwalkers':nwalkers,
             'nit':nit,
             'ncut':ncut,
             # 'usr_log_prior':usr_log_prior, # provided at runtime
             'n_mass':n_mass,
             'int_r_range':int_r_range,
             'iso':iso,
             'iso_filename':iso_grid_filename,
             'jkmins':jkmins,
             # 'feh_range':feh_range, # provided at runtime
             'logg_range':logg_range,
             'fit_dir':fit_dir,
             'gap_dir':gap_dir,
             'ksf_dir':ksf_dir,
             # 'version':version, # provided at runtime
             'verbose':verbose,
             'ro':ro,
             'vo':vo,
             'zo':zo}

## pmass.fit() function kwargs
# nprocs set in config file
force_fit = True
mle_init = True
just_mle = False
return_walkers = True
optimizer_method = 'Powell'
mass_int_type = 'spherical_grid'
batch_masses = True
make_ml_aic_bic = True
calculate_masses = True
post_optimization = True
mcmc_diagnostic = True

fit_kwargs = {# 'nprocs':nprocs, # Normally given at runtime 
              'force_fit':force_fit,
              'mle_init':mle_init,
              'just_mle':just_mle,
              'return_walkers':return_walkers,
              'optimizer_method':optimizer_method,
              'mass_int_type':mass_int_type,
              'batch_masses':batch_masses,
              'make_ml_aic_bic':make_ml_aic_bic,
              'calculate_masses':calculate_masses,
              'post_optimization':post_optimization,
              'mcmc_diagnostic':mcmc_diagnostic,
              }

# Fits to GS/E halo data. Use -2 < [Fe/H] < -0.6

## Fits to data with e-Lz selection, GS/E [Fe/H] range

In [ ]:
densfunc = pdens.triaxial_single_angle_zvecpa
selec = ['eLz',]
fit_type = 'gse'
version = '100w_1e4n'

hf = pmass.HaloFit(densfunc=densfunc,
                   selec=selec,
                   fit_type=fit_type,
                   feh_range=feh_range_fit,
                   version=version,
                   **hf_kwargs)
print(hf.n_star)

### Triaxial Single power law

In [ ]:
def usr_log_prior(densfunc,params):
    return 0

densfunc = pdens.triaxial_single_angle_zvecpa
hf.set_densfunc(densfunc,usr_log_prior=usr_log_prior)
pmass.fit(hf, nprocs=nprocs, **fit_kwargs)

densfunc = pdens.triaxial_single_angle_zvecpa_plusexpdisk
hf.set_densfunc(densfunc,usr_log_prior=usr_log_prior)
pmass.fit(hf, nprocs=nprocs, **fit_kwargs)

### Triaxial exponential cutoff

In [ ]:
def usr_log_prior(densfunc,params):
    assert densfunc is pdens.triaxial_single_cutoff_zvecpa_plusexpdisk\
        or densfunc is pdens.triaxial_single_cutoff_zvecpa
    if params[1] < 2.: return -np.inf
    if params[1] > 55.: return -np.inf
    return 0

densfunc = pdens.triaxial_single_cutoff_zvecpa
hf.set_densfunc(densfunc,usr_log_prior=usr_log_prior)
pmass.fit(hf, nprocs=nprocs, **fit_kwargs)

densfunc = pdens.triaxial_single_cutoff_zvecpa_plusexpdisk
hf.set_densfunc(densfunc,usr_log_prior=usr_log_prior)
pmass.fit(hf, nprocs=nprocs, **fit_kwargs)

### Triaxial broken power law

In [ ]:
def usr_log_prior(densfunc,params):
    assert densfunc is pdens.triaxial_broken_angle_zvecpa_plusexpdisk\
        or densfunc is pdens.triaxial_broken_angle_zvecpa
    if params[0] > 10.: return -np.inf
    if params[1] > 10.: return -np.inf
    if params[2] < 2.: return -np.inf
    if params[2] > 55.: return -np.inf
    return 0

densfunc = pdens.triaxial_broken_angle_zvecpa
hf.set_densfunc(densfunc,usr_log_prior=usr_log_prior)
pmass.fit(hf, nprocs=nprocs, **fit_kwargs)

densfunc = pdens.triaxial_broken_angle_zvecpa_plusexpdisk
hf.set_densfunc(densfunc,usr_log_prior=usr_log_prior)
pmass.fit(hf, nprocs=nprocs, **fit_kwargs)

### Triaxial double broken power law

In [ ]:
def usr_log_prior(densfunc,params):
    # return 0
    assert densfunc is pdens.triaxial_double_broken_angle_zvecpa_plusexpdisk\
        or densfunc is pdens.triaxial_double_broken_angle_zvecpa
    if params[0] > 10.: return -np.inf
    if params[1] > 10.: return -np.inf
    if params[2] > 10.: return -np.inf
    #if params[3] < 2.: return -np.inf
    if params[3] > 55.: return -np.inf
    #if params[4] < 2.: return -np.inf
    if params[4] > 55.: return -np.inf
    return 0

densfunc = pdens.triaxial_double_broken_angle_zvecpa
hf.set_densfunc(densfunc,usr_log_prior=usr_log_prior)
pmass.fit(hf, nprocs=nprocs, **fit_kwargs)

densfunc = pdens.triaxial_double_broken_angle_zvecpa_plusexpdisk
hf.set_densfunc(densfunc,usr_log_prior=usr_log_prior)
pmass.fit(hf, nprocs=nprocs, **fit_kwargs)

## Fits to data with AD selection, GS/E [Fe/H] range

In [ ]:
selec = ['AD',]
densfunc = pdens.triaxial_single_angle_zvecpa
hf = pmass.HaloFit(densfunc=densfunc,
                   selec=selec,
                   fit_type=fit_type,
                   feh_range=feh_range_fit,
                   version=version,
                   **hf_kwargs)
print(hf.n_star)

### Triaxial single power law

In [ ]:
def usr_log_prior(densfunc,params):
    return 0

densfunc = pdens.triaxial_single_angle_zvecpa
hf.set_densfunc(densfunc,usr_log_prior=usr_log_prior)
pmass.fit(hf, nprocs=nprocs, **fit_kwargs)

densfunc = pdens.triaxial_single_angle_zvecpa_plusexpdisk
hf.set_densfunc(densfunc,usr_log_prior=usr_log_prior)
pmass.fit(hf, nprocs=nprocs, **fit_kwargs)

### Triaxial exponential cutoff

In [ ]:
def usr_log_prior(densfunc,params):
    assert densfunc is pdens.triaxial_single_cutoff_zvecpa_plusexpdisk\
        or densfunc is pdens.triaxial_single_cutoff_zvecpa
    if params[1] < 2.: return -np.inf
    if params[1] > 55.: return -np.inf
    return 0

densfunc = pdens.triaxial_single_cutoff_zvecpa
hf.set_densfunc(densfunc,usr_log_prior=usr_log_prior)
pmass.fit(hf, nprocs=nprocs, **fit_kwargs)

densfunc = pdens.triaxial_single_cutoff_zvecpa_plusexpdisk
hf.set_densfunc(densfunc,usr_log_prior=usr_log_prior)
pmass.fit(hf, nprocs=nprocs, **fit_kwargs)

### Triaxial broken power law

In [ ]:
def usr_log_prior(densfunc,params):
    assert densfunc is pdens.triaxial_broken_angle_zvecpa_plusexpdisk\
        or densfunc is pdens.triaxial_broken_angle_zvecpa
    if params[0] > 10.: return -np.inf
    if params[1] > 10.: return -np.inf
    if params[2] < 2.: return -np.inf
    if params[2] > 55.: return -np.inf
    return 0

densfunc = pdens.triaxial_broken_angle_zvecpa
hf.set_densfunc(densfunc,usr_log_prior=usr_log_prior)
pmass.fit(hf, nprocs=nprocs, **fit_kwargs)

densfunc = pdens.triaxial_broken_angle_zvecpa_plusexpdisk
hf.set_densfunc(densfunc,usr_log_prior=usr_log_prior)
pmass.fit(hf, nprocs=nprocs, **fit_kwargs)

In [ ]:
# nit = int(2e4)

# hf = pmass.HaloFit(allstar=allstar_nomask, orbs=orbs_nomask, densfunc=densfunc,
#     nwalkers=nwalkers, nit=nit, ncut=ncut, selec=selec, 
#     usr_log_prior=usr_log_prior, n_mass=n_mass, effsel=apof, 
#     effsel_grid=effsel_grid, effsel_mask=apogee_effSF_mask, 
#     iso_filename=iso_grid_filename, int_r_range=int_r_range, 
#     feh_range=feh_range_gse, logg_range=logg_range, jkmins=jkmins, init=None, 
#     init_type='ML', fit_dir=fit_dir, gap_dir=gap_dir, ksf_dir=ksf_dir, 
#     fit_type=fit_type, version=version, verbose=verbose, ro=ro, vo=vo, zo=zo)

# def usr_log_prior(densfunc,params):
#     # return 0
#     assert densfunc is pdens.triaxial_broken_angle_zvecpa_plusexpdisk\
#         or densfunc is pdens.triaxial_broken_angle_zvecpa\
#         or densfunc is pdens.triaxial_broken_angle_zvecpa_inv
#     if params[0] > 10.: return -np.inf
#     if params[1] > 10.: return -np.inf
#     if params[2] > 1.: return -np.inf
#     return 0

# densfunc = pdens.triaxial_broken_angle_zvecpa_inv
# hf.set_densfunc(densfunc,usr_log_prior=usr_log_prior)
# fit_data(hf, nprocs=nprocs, **fit_data_kwargs)

# # densfunc = pdens.triaxial_broken_angle_zvecpa_plusexpdisk
# # hf.set_densfunc(densfunc,usr_log_prior=usr_log_prior)
# # fit_data(hf, nprocs=nprocs, **fit_data_kwargs)

# nit = int(1e6)

# hf = pmass.HaloFit(allstar=allstar_nomask, orbs=orbs_nomask, densfunc=densfunc,
#     nwalkers=nwalkers, nit=nit, ncut=ncut, selec=selec, 
#     usr_log_prior=usr_log_prior, n_mass=n_mass, effsel=apof, 
#     effsel_grid=effsel_grid, effsel_mask=apogee_effSF_mask, 
#     iso_filename=iso_grid_filename, int_r_range=int_r_range, 
#     feh_range=feh_range_gse, logg_range=logg_range, jkmins=jkmins, init=None, 
#     init_type='ML', fit_dir=fit_dir, gap_dir=gap_dir, ksf_dir=ksf_dir, 
#     fit_type=fit_type, version=version, verbose=verbose, ro=ro, vo=vo, zo=zo)

### Triaxial double broken power law

In [ ]:
def usr_log_prior(densfunc,params):
    # return 0
    assert densfunc is pdens.triaxial_double_broken_angle_zvecpa_plusexpdisk\
        or densfunc is pdens.triaxial_double_broken_angle_zvecpa
    if params[0] > 10.: return -np.inf
    if params[1] > 10.: return -np.inf
    if params[2] > 10.: return -np.inf
    #if params[3] < 2.: return -np.inf
    if params[3] > 55.: return -np.inf
    #if params[4] < 2.: return -np.inf
    if params[4] > 55.: return -np.inf
    return 0

densfunc = pdens.triaxial_double_broken_angle_zvecpa
hf.set_densfunc(densfunc,usr_log_prior=usr_log_prior)
pmass.fit(hf, nprocs=nprocs, **fit_kwargs)

densfunc = pdens.triaxial_double_broken_angle_zvecpa_plusexpdisk
hf.set_densfunc(densfunc,usr_log_prior=usr_log_prior)
pmass.fit(hf, nprocs=nprocs, **fit_kwargs)

## Fits to data with JRLz selection, GS/E [Fe/H] range

In [ ]:
selec = ['JRLz',]
densfunc = pdens.triaxial_single_angle_zvecpa
hf = pmass.HaloFit(densfunc=densfunc,
                   selec=selec,
                   fit_type=fit_type,
                   feh_range=feh_range_fit,
                   version=version,
                   **hf_kwargs)
print(hf.n_star)

### Triaxial single power law

In [ ]:
def usr_log_prior(densfunc,params):
    return 0

densfunc = pdens.triaxial_single_angle_zvecpa
hf.set_densfunc(densfunc,usr_log_prior=usr_log_prior)
pmass.fit(hf, nprocs=nprocs, **fit_kwargs)

densfunc = pdens.triaxial_single_angle_zvecpa_plusexpdisk
hf.set_densfunc(densfunc,usr_log_prior=usr_log_prior)
pmass.fit(hf, nprocs=nprocs, **fit_kwargs)

### Triaxial exponential cutoff

In [ ]:
def usr_log_prior(densfunc,params):
    assert densfunc is pdens.triaxial_single_cutoff_zvecpa_plusexpdisk\
        or densfunc is pdens.triaxial_single_cutoff_zvecpa
    if params[1] < 2.: return -np.inf
    if params[1] > 55.: return -np.inf
    return 0

densfunc = pdens.triaxial_single_cutoff_zvecpa
hf.set_densfunc(densfunc,usr_log_prior=usr_log_prior)
pmass.fit(hf, nprocs=nprocs, **fit_kwargs)

densfunc = pdens.triaxial_single_cutoff_zvecpa_plusexpdisk
hf.set_densfunc(densfunc,usr_log_prior=usr_log_prior)
pmass.fit(hf, nprocs=nprocs, **fit_kwargs)

### Triaxial broken power law

In [ ]:
def usr_log_prior(densfunc,params):
    assert densfunc is pdens.triaxial_broken_angle_zvecpa_plusexpdisk\
        or densfunc is pdens.triaxial_broken_angle_zvecpa
    if params[0] > 10.: return -np.inf
    if params[1] > 10.: return -np.inf
    if params[2] < 2.: return -np.inf
    if params[2] > 55.: return -np.inf
    return 0

densfunc = pdens.triaxial_broken_angle_zvecpa
hf.set_densfunc(densfunc,usr_log_prior=usr_log_prior)
pmass.fit(hf, nprocs=nprocs, **fit_kwargs)

densfunc = pdens.triaxial_broken_angle_zvecpa_plusexpdisk
hf.set_densfunc(densfunc,usr_log_prior=usr_log_prior)
pmass.fit(hf, nprocs=nprocs, **fit_kwargs)

In [ ]:
# nit = int(2e4)

# hf = pmass.HaloFit(allstar=allstar_nomask, orbs=orbs_nomask, densfunc=densfunc,
#     nwalkers=nwalkers, nit=nit, ncut=ncut, selec=selec, 
#     usr_log_prior=usr_log_prior, n_mass=n_mass, effsel=apof, 
#     effsel_grid=effsel_grid, effsel_mask=apogee_effSF_mask, 
#     iso_filename=iso_grid_filename, int_r_range=int_r_range, 
#     feh_range=feh_range_gse, logg_range=logg_range, jkmins=jkmins, init=None, 
#     init_type='ML', fit_dir=fit_dir, gap_dir=gap_dir, ksf_dir=ksf_dir, 
#     fit_type=fit_type, version=version, verbose=verbose, ro=ro, vo=vo, zo=zo)

# def usr_log_prior(densfunc,params):
#     # return 0
#     assert densfunc is pdens.triaxial_broken_angle_zvecpa_plusexpdisk\
#         or densfunc is pdens.triaxial_broken_angle_zvecpa\
#         or densfunc is pdens.triaxial_broken_angle_zvecpa_inv
#     if params[0] > 10.: return -np.inf
#     if params[1] > 10.: return -np.inf
#     if params[2] > 1.: return -np.inf
#     return 0

# densfunc = pdens.triaxial_broken_angle_zvecpa_inv
# hf.set_densfunc(densfunc,usr_log_prior=usr_log_prior)
# fit_data(hf, nprocs=nprocs, **fit_data_kwargs)

# # densfunc = pdens.triaxial_broken_angle_zvecpa_plusexpdisk
# # hf.set_densfunc(densfunc,usr_log_prior=usr_log_prior)
# # fit_data(hf, nprocs=nprocs, **fit_data_kwargs)

# nit = int(1e6)

# hf = pmass.HaloFit(allstar=allstar_nomask, orbs=orbs_nomask, densfunc=densfunc,
#     nwalkers=nwalkers, nit=nit, ncut=ncut, selec=selec, 
#     usr_log_prior=usr_log_prior, n_mass=n_mass, effsel=apof, 
#     effsel_grid=effsel_grid, effsel_mask=apogee_effSF_mask, 
#     iso_filename=iso_grid_filename, int_r_range=int_r_range, 
#     feh_range=feh_range_gse, logg_range=logg_range, jkmins=jkmins, init=None, 
#     init_type='ML', fit_dir=fit_dir, gap_dir=gap_dir, ksf_dir=ksf_dir, 
#     fit_type=fit_type, version=version, verbose=verbose, ro=ro, vo=vo, zo=zo)

### Triaxial double broken power law

In [ ]:
def usr_log_prior(densfunc,params):
    # return 0
    assert densfunc is pdens.triaxial_double_broken_angle_zvecpa_plusexpdisk\
        or densfunc is pdens.triaxial_double_broken_angle_zvecpa
    if params[0] > 10.: return -np.inf
    if params[1] > 10.: return -np.inf
    if params[2] > 10.: return -np.inf
    #if params[3] < 2.: return -np.inf
    if params[3] > 55.: return -np.inf
    #if params[4] < 2.: return -np.inf
    if params[4] > 55.: return -np.inf
    return 0

densfunc = pdens.triaxial_double_broken_angle_zvecpa
hf.set_densfunc(densfunc,usr_log_prior=usr_log_prior)
pmass.fit(hf, nprocs=nprocs, **fit_kwargs)

densfunc = pdens.triaxial_double_broken_angle_zvecpa_plusexpdisk
hf.set_densfunc(densfunc,usr_log_prior=usr_log_prior)
pmass.fit(hf, nprocs=nprocs, **fit_kwargs)